In [ ]:
import copy
import numpy as np
import sklearn
import pandas as pd
import matplotlib.pyplot as plt
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import LSTM
from keras.layers import Bidirectional
from keras.layers import RepeatVector
from keras.layers import TimeDistributed
from numpy import hstack
from sklearn.preprocessing import StandardScaler
import datetime
import time
import joblib
from datetime import timedelta, date
import tensorflow as tf
from sklearn.preprocessing import MinMaxScaler
from numpy import array
import os
import seaborn as sns; sns.set_theme() 
import errno

In [ ]:
url = "https://raw.githubusercontent.com/rohitash-chandra/CMTL_dynamictimeseries/master/IndianOcean/rawtrain1985-2001.txt"
df = pd.read_csv(url, sep = "\t", header = None)
df.columns = ['id','date','longitude','latitude','speed']
df = df.drop(['date'], axis = 1)
df.to_csv('adjusted.csv')
df

,id,longitude,latitude,speed
0,1,7.7,80.9,20
1,1,8.1,79.9,20
2,1,8.5,78.8,25
3,1,8.8,77.7,25
4,1,9.0,76.6,30
...,...,...,...,...
9360,21,25.9,38.9,35
9361,21,25.9,40.2,35
9362,21,25.5,41.0,35
9363,21,25.1,41.7,30


In [ ]:
#making starting point of every cyclone same, spatial bias removed
id = 1
x0 = df['longitude'][0]
y0 = df['latitude'][0]
df['longitude'][0] = 0
df['latitude'][0] = 0
for i in range(1, df.shape[0]):
  if df['id'][i] == id :
    df['longitude'][i] = df['longitude'][i] - x0
    df['latitude'][i] = df['latitude'][i] - y0
  else:
    x0 = df['longitude'][i]
    y0 = df['latitude'][i]
    df['longitude'][i] = 0
    df['latitude'][i] = 0
    id = df['id'][i]

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  if __name__ == '__main__':
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:10: SettingWithC

In [ ]:
longi = array(df['longitude'])
lat = array(df['latitude'])
speed = array(df['speed'])
longi = longi.reshape((len(longi), 1))
lat = lat.reshape((len(lat), 1))
speed = speed.reshape((len(speed), 1))
longi.shape, lat.shape, speed.shape

((9365, 1), (9365, 1), (9365, 1))

In [ ]:
sc = StandardScaler()
longi_scaled = sc.fit_transform(longi)
lat_scaled = sc.fit_transform(lat)
speed_scaled = sc.fit_transform(speed)
speed_scaled

array([[-0.98138094],
       [-0.98138094],
       [-0.78866885],
       ...,
       [-0.40324466],
       [-0.59595675],
       [-0.78866885]])

In [ ]:
#for univariate
def split_sequence(sequence, n_steps_in, n_steps_out):
	X, y = list(), list()
	for i in range(len(sequence)):
		# find the end of this pattern
		end_ix = i + n_steps_in
		out_end_ix = end_ix + n_steps_out
		# check if we are beyond the sequence
		if out_end_ix > len(sequence):
			break
		# gather input and output parts of the pattern
		seq_x, seq_y = sequence[i:end_ix], sequence[end_ix:out_end_ix]
		X.append(seq_x)
		y.append(seq_y)
	return np.array(X), np.array(y)
 
def rmse(pred, actual):
	return np.sqrt(((pred - actual) ** 2).mean())

In [ ]:
univariate = True # if false, its multivariate case
n_steps_in = 5
n_steps_out = 3
n_features = 1
Hidden = 10
Epochs = 50
Num_Exp = 10
dataset = hstack((longi_scaled, lat_scaled, speed_scaled))
dataset.shape

(9365, 3)

In [ ]:
if univariate is True:  
  train = speed_scaled[0:8628]
  test = speed_scaled[8629:9364]
  x_train, y_train = split_sequence(train, n_steps_in, n_steps_out)
  x_test, y_test = split_sequence(test, n_steps_in, n_steps_out)
  x_train = x_train.reshape((x_train.shape[0], x_train.shape[1], n_features))
  print(x_train.shape)
  x_test = x_test.reshape((x_test.shape[0], x_test.shape[1], n_features))
  print(x_test.shape)
  print(y_train.shape)
  y_train = y_train.reshape((y_train.shape[0],y_train.shape[1]))
  print(y_train.shape)
  y_test = y_test.reshape((y_test.shape[0],y_test.shape[1]))
  print(y_test.shape)

else:
  train = dataset[0:8628]
  test = dataset[8629:9364]
  x_train, y_train = msplit_sequence(train, n_steps_in, n_steps_out)
  x_test, y_test = msplit_sequence(test, n_steps_in, n_steps_out)
  x_train = x_train.reshape((x_train.shape[0], x_train.shape[1], n_features))
  print(x_train.shape)
  x_test = x_test.reshape((x_test.shape[0], x_test.shape[1], n_features))
  print(x_test.shape)
  y_train = y_train.reshape((y_train.shape[0]))
  print(y_train.shape)
  y_test = y_test.reshape((y_test.shape[0]))
  print(y_test.shape)


(8621, 5, 1)
(728, 5, 1)
(8621, 3, 1)
(8621, 3)
(728, 3)


In [ ]:
model = Sequential()

def MODEL_LSTM(univariate, x_train, x_test, y_train, y_test, Num_Exp, n_steps_in, n_steps_out, Epochs, Hidden):

  train_acc = np.zeros(Num_Exp)
  test_acc = np.zeros(Num_Exp)
 
  model.add(LSTM(Hidden, activation='relu', input_shape=(n_steps_in, n_features), dropout=0.2))
  # model.add(Dense(Hidden)) (No particular need of this)
  model.add(Dense(n_steps_out))
  model.compile(optimizer='adam', loss='mse')
  model.summary()

  y_predicttest_allruns = np.zeros([Num_Exp, x_test.shape[0], n_steps_out])

  print(y_predicttest_allruns.shape, ' shape ')


  Best_RMSE = 1000  # Assigning a large number

  start_time = time.time()
  for run in range(Num_Exp):
    print("Experiment", run + 1, "in progress")
    # fit model
    model.fit(x_train, y_train, epochs=Epochs, batch_size=10, verbose=0, shuffle=False)
    y_predicttrain = model.predict(x_train)
    y_predicttest = model.predict(x_test)
    y_predicttest_allruns[run,:,:] = y_predicttest
    train_acc[run] = rmse(y_predicttrain, y_train)
    print(train_acc[run], 'train accuracy')
    test_acc[run] = rmse(y_predicttest, y_test)
    if test_acc[run] < Best_RMSE:
      Best_RMSE = test_acc[run]
      Best_Predict_Test = y_predicttest

  print("Total time for", Num_Exp, "experiments", time.time() - start_time)
  return train_acc, test_acc, Best_Predict_Test, y_predicttrain, y_predicttest, y_predicttest_allruns

In [ ]:
train_acc, test_acc,Best_Predict_Test, y_predicttrain, y_predicttest, y_predicttest_allruns = MODEL_LSTM(univariate,x_train,x_test,y_train,y_test,Num_Exp,n_steps_in,n_steps_out,Epochs, Hidden)


Model: "sequential_5"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_8 (LSTM)                (None, 10)                480       
_________________________________________________________________
dense_5 (Dense)              (None, 3)                 33        
Total params: 513
Trainable params: 513
Non-trainable params: 0
_________________________________________________________________
(2, 728, 3)  shape 
Experiment 1 in progress
0.3685741749734835 train accuracy
Experiment 2 in progress
0.3608747686976727 train accuracy
Total time for 2 experiments 197.0345594882965


In [ ]:
print(train_acc, test_acc) 
mean_train = np.mean(train_acc, axis=0)
mean_test = np.mean(test_acc, axis=0)
print(mean_train, 'mean rmse train') 
print(mean_test, 'mean rmse test')

[0.36857417 0.36087477] [0.44055375 0.43570608]
0.3647244718355781 mean rmse train
0.4381299127685311 mean rmse test


In [ ]:
y_predicttrain.shape

(8626, 1)

In [ ]:
y_train.shape

(8626, 1)

In [ ]:
dataset

array([[-0.94358799,  0.66878293, -0.98138094],
       [-0.86136489,  0.58367639, -0.98138094],
       [-0.77914179,  0.49005919, -0.78866885],
       ...,
       [-1.86859786,  1.24750741, -0.40324466],
       [-1.95082096,  1.30708199, -0.59595675],
       [-2.05359984,  1.3241033 , -0.78866885]])

In [ ]:
x_train

array([[[-0.94358799,  0.66878293],
        [-0.86136489,  0.58367639],
        [-0.77914179,  0.49005919]],

       [[-0.86136489,  0.58367639],
        [-0.77914179,  0.49005919],
        [-0.71747447,  0.39644199]],

       [[-0.77914179,  0.49005919],
        [-0.71747447,  0.39644199],
        [-0.67636292,  0.3028248 ]],

       ...,

       [[-0.18302432,  1.06027302],
        [-0.03913389,  1.04325171],
        [ 0.02253343,  0.99218779]],

       [[-0.03913389,  1.04325171],
        [ 0.02253343,  0.99218779],
        [ 0.08420076,  0.94963452]],

       [[ 0.02253343,  0.99218779],
        [ 0.08420076,  0.94963452],
        [ 0.18697963,  0.9155919 ]]])